In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
import os
#import matplotlib.pyplot as plt
#import pandas as pd
#import sklearn
from sklearn.model_selection import train_test_split
from torch.utils.data import WeightedRandomSampler
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import torchvision.models as models
from tqdm import tqdm
import copy
import os
import numpy as np
from PIL import Image
import torch
from torchvision import models, transforms
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
#from collections import Counter, defaultdict
import matplotlib.pyplot as plt
#import shutil
#import random
import pandas as pd


# Model resnet50 na danych z folderu cropped_out_50 z użyciem klasteryzacji

In [ ]:
def save_to_list(path):
    output_root = path
    X = []
    y = []
    l=[]
        
    label_to_index = {
        'AVM': 0,
        'Normal': 1,
        'Ulcer': 2
    }
    print(label_to_index)
    for root, _, files in os.walk(output_root):
        for file in files:
            if file.endswith(".bmp"): 
                input_path = os.path.join(root, file)

                base, ext = os.path.splitext(file)
                new_filename = f"{base}{ext}"
                image = cv2.imread(input_path)

                X.append(image)
                y.append(label_to_index[new_filename.split("_")[0]])
                l.append(new_filename)
    return X, y, l

X, y, l = save_to_list("cropped_out_50")

{'AVM': 0, 'Normal': 1, 'Ulcer': 2}


In [14]:
# https://discuss.pytorch.org/t/balanced-sampling-between-classes-with-torchvision-dataloader/2703/2

class ImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform 
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])  
        label = torch.tensor(self.labels[idx], dtype=torch.long) 

        if self.transform:
            image = self.transform(image)
            
        
        return image, label

In [15]:
class ResNet50Transfer(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet50Transfer, self).__init__()
        self.base_model = models.resnet50(pretrained=True)

        # Freeze all layers (optional: you can unfreeze later for fine-tuning)
        for param in self.base_model.parameters():
            param.requires_grad = False
        
        for name, param in self.base_model.named_parameters():
            if "layer4" in name or "layer3" in name:  # Unfreeze only final block
                param.requires_grad = True

        in_features = self.base_model.fc.in_features  # Usually 2048
        self.base_model.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, num_classes) 
        )

    def forward(self, x):
        return self.base_model(x)


In [16]:
def train_and_evaluate_full(model, train_loader, test_loader, save_path, epochs=15, learning_rate=0.001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    print(f"Using device: {device}") 
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

    best_test_loss = float('inf')
    best_model_weights = copy.deepcopy(model.state_dict())

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
        all_labels = []
        all_predictions = []

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

        train_loss = running_loss / total
        train_acc = correct / total
        train_precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
        train_recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
        train_f1 = f1_score(all_labels, all_predictions, average='weighted', zero_division=0)

        # Evaluation on test set
        model.eval()
        test_loss = 0.0
        test_correct, test_total = 0, 0
        test_labels, test_predictions = [], []

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                test_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                test_correct += (predicted == labels).sum().item()
                test_total += labels.size(0)
                test_labels.extend(labels.cpu().numpy())
                test_predictions.extend(predicted.cpu().numpy())

        test_loss /= test_total
        test_acc = test_correct / test_total
        test_precision = precision_score(test_labels, test_predictions, average='weighted', zero_division=0)
        test_recall = recall_score(test_labels, test_predictions, average='weighted', zero_division=0)
        test_f1 = f1_score(test_labels, test_predictions, average='weighted', zero_division=0)

        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}")
        print(f"Test  Loss: {test_loss:.4f}, Acc: {test_acc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}")

        scheduler.step()

        # Save best model based on test loss
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_model_weights = copy.deepcopy(model.state_dict())
            print("Best model updated!")

    # Load best model for final evaluation
    model.load_state_dict(best_model_weights)
    torch.save(model.state_dict(), save_path)
    print(f"Best model saved as {save_path}'") # rozszerzenie .pth

    # Final evaluation on test set
    model.eval()
    correct, total = 0, 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    test_acc = correct / total
    test_precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
    test_recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
    test_f1 = f1_score(all_labels, all_predictions, average='weighted', zero_division=0)
    test_conf_matrix = confusion_matrix(all_labels, all_predictions)

    print("\n Final Test Evaluation:")
    print(f"Accuracy: {test_acc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}")
    print("Confusion Matrix:\n", test_conf_matrix)

    return model


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),          
    transforms.RandomRotation((-180,180)),  
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),                  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])  # do testów - bez zmian

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def extract_image_features(images):
    features = []
    for img in images:
        img_resized = cv2.resize(img, (224, 224))  
        img_flattened = img_resized.flatten()  
        features.append(img_flattened)
    return np.array(features)

image_features = extract_image_features(X) 
db = DBSCAN(eps=6.5, min_samples=5)
clusters = db.fit_predict(image_features)

# Podział na zbiory treningowy i testowy
train_indices, temp_indices = train_test_split(range(len(X)), test_size=0.2, stratify=y, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, stratify=[y[i] for i in temp_indices], random_state=42)

# Wybór danych dla poszczególnych zbiorów
train_images = [X[i] for i in train_indices]
train_labels = [y[i] for i in train_indices]
val_images = [X[i] for i in val_indices]
val_labels = [y[i] for i in val_indices]
test_images = [X[i] for i in test_indices]
test_labels = [y[i] for i in test_indices]


class_counts = np.bincount(train_labels)
weights = 1. / class_counts  # Wagi odwrotności częstotliwości
sample_weights = np.array([weights[label] for label in train_labels])

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

train_dataset = ImageDataset(train_images, train_labels, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)

test_dataset = ImageDataset(test_images, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [18]:
model = ResNet50Transfer(num_classes=3)
trained_model = train_and_evaluate_full(model, train_loader, test_loader, "model_cropped_photos_radius_50.pth")

c:\Users\alapr\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alapr\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using device: cuda


Epoch 1/15: 100%|██████████| 83/83 [00:13<00:00,  6.16it/s]


Epoch 1/15
Train Loss: 0.5586, Acc: 0.7759, Precision: 0.7807, Recall: 0.7759, F1: 0.7764
Test  Loss: 0.4032, Acc: 0.7994, Precision: 0.8325, Recall: 0.7994, F1: 0.7868
Best model updated!


Epoch 2/15: 100%|██████████| 83/83 [00:13<00:00,  6.25it/s]


...

Epoch 13/15
Train Loss: 0.1383, Acc: 0.9509, Precision: 0.9514, Recall: 0.9509, F1: 0.9509
Test  Loss: 0.1621, Acc: 0.9422, Precision: 0.9428, Recall: 0.9422, F1: 0.9421
Best model updated!


Epoch 14/15: 100%|██████████| 83/83 [00:12<00:00,  6.40it/s]


Epoch 14/15
Train Loss: 0.1578, Acc: 0.9399, Precision: 0.9402, Recall: 0.9399, F1: 0.9399
Test  Loss: 0.1588, Acc: 0.9514, Precision: 0.9519, Recall: 0.9514, F1: 0.9512
Best model updated!


Epoch 15/15: 100%|██████████| 83/83 [00:12<00:00,  6.42it/s]


Epoch 15/15
Train Loss: 0.2238, Acc: 0.9224, Precision: 0.9234, Recall: 0.9224, F1: 0.9226
Test  Loss: 0.1813, Acc: 0.9362, Precision: 0.9422, Recall: 0.9362, F1: 0.9373
Best model saved as model_cropped_photos_radius_50.pth'

 Final Test Evaluation:
Accuracy: 0.9514, Precision: 0.9519, Recall: 0.9514, F1: 0.9512
Confusion Matrix:
 [[ 59   6   2]
 [  1 210   4]
 [  2   1  44]]
